# Part VI — Async, Realtime, and Background Work  
## 27. ASGI, Async Views, and Django’s Async Capabilities (What’s Real, What’s Not, How to Use It Safely)

Django supports **asynchronous (“async”) views** and can run an **async-enabled
request stack** when deployed under **ASGI**. Async views also run under **WSGI**,
but with important limitations and overhead.

This chapter teaches you the practical, industry-grade truth:

- Async is not “make everything faster.”
- Async helps primarily with **I/O-bound concurrency** (calling other services,
  long-polling, slow streaming), not CPU-heavy work.
- You only get the full benefit of an async stack if your middleware stack is
  compatible (no sync middleware forcing thread-per-request emulation).
- Django’s ORM has **async variants** for query-evaluating operations; however,
  **transactions still don’t work in async mode** (per Django async docs), so you
  need patterns for “transactional sync islands.”

References (official Django docs):
- Asynchronous support: https://docs.djangoproject.com/en/6.0/topics/async/
- Deploy with ASGI: https://docs.djangoproject.com/en/6.0/howto/deployment/asgi/

---

## 27.0 Learning Outcomes

By the end, you should be able to:

1. Explain WSGI vs ASGI with a correct mental model and deployment implications.
2. Write async FBVs and async CBVs correctly (and know what you must not mix).
3. Use async to run concurrent I/O (e.g., multiple external HTTP calls).
4. Use Django’s async ORM interfaces (`a...` methods, `async for`) safely.
5. Understand and fix `SynchronousOnlyOperation` (async safety protection).
6. Understand why **transactions don’t work in async mode** and how to handle it
   with a “sync island” wrapped by `sync_to_async`.
7. Understand middleware compatibility and why sync middleware can negate async benefits.
8. Run your Django app under an ASGI server (Uvicorn) locally to validate behavior.
9. Write at least one async test using Django’s async test client.

---

## 27.1 WSGI vs ASGI (Accurate Mental Model)

### 27.1.1 WSGI (traditional, synchronous)
WSGI is the older Python standard interface:
- one request handled in a synchronous call stack
- concurrency is usually achieved by:
  - multiple processes (Gunicorn workers)
  - threads (threaded servers)

**Good for:**
- classic server-rendered sites
- purely sync Django apps
- simplicity and maturity

### 27.1.2 ASGI (async-friendly, modern)
ASGI is the newer standard designed for:
- async request handling
- long-lived connections (streaming, long-polling)
- other protocols (WebSockets) in the ecosystem (often with Channels)

Django can run under ASGI using `config/asgi.py`’s `application` object.

**The key promise of ASGI** is not “each request is faster.”
It’s: “the server can handle many concurrent connections efficiently without a
thread per connection.”

---

## 27.2 Async in Django: What You Get (and What You Don’t)

From Django’s async docs:

- Async views work under both WSGI and ASGI.
- Async benefits (servicing many connections, long-running requests efficiently)
  require ASGI.
- If you have any synchronous-only middleware, Django may need to use a thread per
  request to emulate a synchronous environment, reducing the async advantage.
- ORM has async query methods, but **transactions do not yet work in async mode**.
- Persistent DB connections (`CONN_MAX_AGE`) should be disabled in async mode
  (Django docs recommend this).

---

## 27.3 Async Views (Function-Based) — Correct Patterns

### 27.3.1 The minimal async view
Add to `pages/views.py`:

```python
import asyncio
from django.http import JsonResponse
from django.views.decorators.http import require_GET


@require_GET
async def async_hello(request):
    await asyncio.sleep(0.1)
    return JsonResponse({"message": "hello from async view"})
```

Wire in `pages/urls.py`:

```python
path("async-hello/", views.async_hello, name="async_hello"),
```

#### What this proves
- Django correctly detects an async view (`async def`)
- `await` works
- Response is returned normally

### 27.3.2 Async concurrency: doing multiple I/O operations in parallel
Async is valuable when you have multiple I/O waits.

Add to `pages/views.py`:

```python
import asyncio
import time
from django.http import JsonResponse
from django.views.decorators.http import require_GET


@require_GET
async def async_concurrency_demo(request):
    async def slow_io(n: int) -> dict:
        await asyncio.sleep(0.2)
        return {"task": n, "done": True}

    start = time.perf_counter()

    results = await asyncio.gather(
        slow_io(1),
        slow_io(2),
        slow_io(3),
        slow_io(4),
        slow_io(5),
    )

    duration_ms = (time.perf_counter() - start) * 1000

    return JsonResponse(
        {"mode": "concurrent", "duration_ms": round(duration_ms, 2), "results": results}
    )
```

Add a sequential version too (to compare):

```python
@require_GET
async def async_sequential_demo(request):
    async def slow_io(n: int) -> dict:
        await asyncio.sleep(0.2)
        return {"task": n, "done": True}

    start = time.perf_counter()

    results = []
    for n in range(1, 6):
        results.append(await slow_io(n))

    duration_ms = (time.perf_counter() - start) * 1000

    return JsonResponse(
        {"mode": "sequential", "duration_ms": round(duration_ms, 2), "results": results}
    )
```

Wire URLs:

```python
path("async-demo/concurrent/", views.async_concurrency_demo, name="async_concurrent"),
path("async-demo/sequential/", views.async_sequential_demo, name="async_sequential"),
```

Now test:

```bash
curl -i http://127.0.0.1:8000/async-demo/sequential/
curl -i http://127.0.0.1:8000/async-demo/concurrent/
```

#### What you should observe
- sequential ~ \(5 \times 0.2 = 1.0\) seconds
- concurrent ~ ~0.2–0.3 seconds

This is the real benefit of async: **concurrent waiting**, not faster CPU work.

---

## 27.4 Async CBVs (Class-Based Views) — Rules You Must Follow

Django CBVs can be async by making the HTTP method handlers async.

Example:

```python
import asyncio
from django.http import JsonResponse
from django.views import View


class AsyncStatusView(View):
    async def get(self, request, *args, **kwargs):
        await asyncio.sleep(0.1)
        return JsonResponse({"status": "ok", "kind": "cbv-async"})
```

URL:

```python
from django.urls import path
from .views import AsyncStatusView

urlpatterns = [
    path("async-status/", AsyncStatusView.as_view(), name="async_status"),
]
```

### Critical CBV rule (from Django docs)
Within a single CBV class, **all user-defined method handlers must be all sync or
all async**. Do not mix `def get()` with `async def post()` in the same class.

If you mix them, Django raises `ImproperlyConfigured`.

---

## 27.5 Running Async Views Under WSGI vs ASGI (Performance Reality)

### 27.5.1 Async view under WSGI
Django will run the async view in a one-off event loop per request.
- Your `await` and `asyncio.gather` concurrency will work.
- But you do not get the “handle many long-lived connections efficiently”
  advantage of a fully async request stack.

### 27.5.2 Sync view under ASGI (and vice versa) costs context switching
If your deployment mode doesn’t match your view type:
- Django adapts (sync↔async)
- there’s a small overhead (Django docs note around a millisecond order)

This is usually fine, but don’t “randomly sprinkle async everywhere.”
Choose async where it provides value.

---

## 27.6 Middleware and the “Fully Async Stack” Requirement (Important)

From Django async docs:

> You only get the benefits of a fully asynchronous request stack if you have
> no synchronous middleware loaded into your site.

### 27.6.1 What happens if you have synchronous middleware?
Django may need to:
- run requests in threads to safely emulate synchronous behavior
- hold a thread per request

That can remove most of the ASGI scalability advantage.

### 27.6.2 How to detect middleware adaptation (debugging tip)
Django can log messages like “Asynchronous handler adapted for middleware …”
when debug logging for `django.request` is enabled.

In `LOGGING`, set `django.request` to DEBUG in dev if you want to inspect adaptation.

---

## 27.7 Writing Middleware That Supports Both Sync and Async (Best Practice)

Django provides decorators to mark middleware compatibility. A practical pattern is
to create middleware that returns either a sync or async function depending on
whether `get_response` is async.

Create `config/async_middleware.py`:

```python
import time
from django.utils.decorators import sync_and_async_middleware


@sync_and_async_middleware
def timing_middleware(get_response):
    if callable(get_response) and getattr(get_response, "__call__", None):
        # Determine if get_response is async by checking if it returns a coroutine.
        # The safest runtime check in Django context is:
        import asgiref.sync

        is_async = asgiref.sync.iscoroutinefunction(get_response)
    else:
        is_async = False

    if is_async:

        async def middleware(request):
            start = time.perf_counter()
            response = await get_response(request)
            response["X-MW-Duration-Ms"] = f"{(time.perf_counter() - start) * 1000:.2f}"
            return response

        return middleware

    def middleware(request):
        start = time.perf_counter()
        response = get_response(request)
        response["X-MW-Duration-Ms"] = f"{(time.perf_counter() - start) * 1000:.2f}"
        return response

    return middleware
```

Register it in `MIDDLEWARE`:

```python
MIDDLEWARE = [
    "django.middleware.security.SecurityMiddleware",
    "config.middleware.RequestIdMiddleware",
    "config.access_log_middleware.AccessLogMiddleware",
    "config.async_middleware.timing_middleware",
    # ...
]
```

#### Why this matters
- avoids repeated sync↔async adaptation
- keeps middleware safe and efficient in either execution mode
- future-proofs your stack as you add more async views

---

## 27.8 Django ORM in Async Code (What Works, What Breaks)

According to Django’s async support docs:

- QuerySet methods that trigger SQL queries have **`a`-prefixed async variants**
  (e.g., `.afirst()`, `.aget()`, `.acreate()`, etc., depending on method).
- You can iterate querysets with `async for`.
- Some model methods have async variants like `asave()` and M2M manager methods
  like `aset()`.

### 27.8.1 Example: async list of article slugs (safe pattern)
Add to `articles/views_async.py` (create this file) or `articles/views.py`:

```python
from django.http import JsonResponse
from django.views.decorators.http import require_GET

from articles.models import Article


@require_GET
async def async_article_slugs(request):
    qs = Article.objects.published().order_by("-published_at", "-created_at")

    slugs = []
    async for a in qs.values_list("slug", flat=True)[:50]:
        slugs.append(a)

    return JsonResponse({"count": len(slugs), "slugs": slugs})
```

Wire in `articles/urls.py`:

```python
from .views_async import async_article_slugs

urlpatterns += [
    path("async/slugs/", async_article_slugs, name="async_slugs"),
]
```

#### What to understand
- `Article.objects.published()` is fine because it’s a QuerySet refinement (no SQL yet).
- SQL executes during `async for` evaluation.
- `values_list(..., flat=True)` yields strings, not model objects, which is efficient.

### 27.8.2 Example: async get first published article
```python
from articles.models import Article

article = await Article.objects.published().afirst()
```

This uses an async query-evaluating method.

### 27.8.3 The big limitation: transactions don’t work in async mode
Django’s async docs explicitly state: **transactions do not yet work in async mode.**

That means:
- `transaction.atomic()` is not available in true async context for “real async”
  transaction behavior.
- If you need atomic operations, write a synchronous function and call it with
  `sync_to_async` (next section).

### 27.8.4 Persistent DB connections warning (`CONN_MAX_AGE`)
Django’s async docs recommend disabling persistent DB connections in async mode:
- set `CONN_MAX_AGE = 0` when running async workloads under ASGI
- use your DB backend’s own pooling or external pooling

In `config/settings/prod.py` (if you’re using ASGI + async DB calls):

```python
CONN_MAX_AGE = 0
```

---

## 27.9 Async Safety Protection and `SynchronousOnlyOperation`

If you accidentally call sync-only Django operations from an async context (thread
with a running event loop), Django can raise:

- `django.core.exceptions.SynchronousOnlyOperation`

### 27.9.1 Common way to trigger it (don’t do this)
In an async view:

```python
# bad example
from articles.models import Article

count = Article.objects.count()  # sync evaluation inside async context
```

If this triggers `SynchronousOnlyOperation`, it means:
- you called sync-only ORM evaluation in an async context

### 27.9.2 Correct fixes
1. Prefer async ORM APIs:
   - use `await ...acount()` if available in your Django version
   - or use async iteration patterns
2. Or isolate sync code and call it with `sync_to_async`.

### 27.9.3 The dangerous escape hatch (don’t use in production)
Django documents an env var:

- `DJANGO_ALLOW_ASYNC_UNSAFE`

This disables safety checks. Django warns: you may cause data corruption if there
is concurrent access. Treat it as a last resort for special environments (like
some interactive shells), not production.

---

## 27.10 “Sync Islands” Inside Async Views (Transactions + Legacy Libraries)

When you need:
- transactions (`transaction.atomic`)
- `select_for_update`
- third-party libraries that are sync-only
- large blocks of existing sync ORM code

Use `sync_to_async` to run that code in a separate thread.

### 27.10.1 Example: publish an article atomically (sync function)
Create `articles/services_publish.py`:

```python
from django.db import transaction
from django.utils import timezone

from articles.models import Article


def publish_article_sync(*, article_id: int) -> Article:
    with transaction.atomic():
        article = Article.objects.select_for_update().get(id=article_id)
        article.status = Article.Status.PUBLISHED
        if article.published_at is None:
            article.published_at = timezone.now()
        article.save()
        return article
```

Now call it from an async view:

```python
from asgiref.sync import sync_to_async
from django.http import JsonResponse
from django.views.decorators.http import require_POST

from articles.services_publish import publish_article_sync


@require_POST
async def async_publish_article(request, article_id: int):
    article = await sync_to_async(publish_article_sync, thread_sensitive=True)(
        article_id=article_id
    )
    return JsonResponse({"status": "ok", "slug": article.slug})
```

#### Why `thread_sensitive=True` matters
Django’s ORM and some internals rely on thread-local state.
`thread_sensitive=True` tells `sync_to_async` to run the function in a dedicated,
consistent thread context, reducing the risk of thread-safety issues.

---

## 27.11 Handling Disconnects and Cancellation (Long-Lived Requests)

For long-lived async requests, Django’s async docs note that if the client
disconnects, your view can receive:

- `asyncio.CancelledError`

Example:

```python
import asyncio
from django.http import JsonResponse


async def long_poll(request):
    try:
        await asyncio.sleep(30)
        return JsonResponse({"status": "done"})
    except asyncio.CancelledError:
        # cleanup if needed
        raise
```

#### Why you should care
If you do:
- long polling
- slow streaming
- waiting on external APIs
…you must assume clients disconnect and your view may be cancelled.

---

## 27.12 Running Django Under ASGI Locally (Uvicorn)

Your Django project includes `config/asgi.py` with an `application` callable.

### 27.12.1 Install Uvicorn (dev)
```bash
python -m pip install uvicorn
```

### 27.12.2 Run your Django ASGI app
From the directory containing `manage.py`:

```bash
python -m uvicorn config.asgi:application --reload
```

Visit:
- `http://127.0.0.1:8000/async-demo/concurrent/`

### 27.12.3 Production pattern: Gunicorn + UvicornWorker
Common deployment pattern:

```bash
python -m pip install gunicorn uvicorn
python -m gunicorn config.asgi:application -k uvicorn.workers.UvicornWorker
```

In real production you put this behind Nginx/load balancer and configure timeouts.

---

## 27.13 Async Testing (So You Can Prove It Works)

Django supports asynchronous tests and an async test client.

### 27.13.1 Example async test (Django TestCase)
Create `pages/tests_async.py`:

```python
from django.test import AsyncClient, TestCase


class AsyncViewsTests(TestCase):
    async def test_async_hello(self):
        client = AsyncClient()
        response = await client.get("/async-hello/")
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.json()["message"], "hello from async view")
```

Run:

```bash
python manage.py test
```

#### What this gives you
- confidence that your async endpoints work in CI
- protection against regressions when you refactor middleware/settings

---

## 27.14 When to Use Async in Django (Decision Checklist)

Use async views when:
- you call multiple external services per request (HTTP, gRPC, etc.)
- you need long-lived requests (long polling, slow streaming)
- you expect many concurrent connections with mostly waiting (chat/notifications are often handled via Channels, next chapters)

Do NOT use async views when:
- your view is mostly ORM work + template rendering (DB is often the bottleneck)
- you need lots of transactions and complex DB updates (keep those sync islands)
- your workload is CPU-bound (use background tasks or scale processes)

A pragmatic industry approach:
- keep most of Django sync
- introduce async only where it clearly improves concurrency and responsiveness

---

## 27.15 Hands-On Lab (Chapter Capstone): Add One “Real” Async Endpoint

### Goal
Create an async endpoint that:
- does concurrent I/O (simulated or real)
- does a small async ORM read
- logs duration and keeps request ID headers

### Step A: Add endpoint `/async-report/`
In `pages/views.py`:

```python
import asyncio
import time
from django.http import JsonResponse
from django.views.decorators.http import require_GET

from articles.models import Article


@require_GET
async def async_report(request):
    start = time.perf_counter()

    async def slow_io(label: str):
        await asyncio.sleep(0.15)
        return label

    io_results = await asyncio.gather(
        slow_io("a"),
        slow_io("b"),
        slow_io("c"),
    )

    # Async ORM read: count first 10 slugs
    slugs = []
    async for slug in Article.objects.published().values_list("slug", flat=True)[:10]:
        slugs.append(slug)

    duration_ms = (time.perf_counter() - start) * 1000

    return JsonResponse(
        {
            "io": io_results,
            "slugs_sample": slugs,
            "duration_ms": round(duration_ms, 2),
        }
    )
```

Wire it:

```python
path("async-report/", views.async_report, name="async_report"),
```

### Step B: Run under Uvicorn and test
```bash
python -m uvicorn config.asgi:application --reload
curl -i http://127.0.0.1:8000/async-report/
```

### Step C: Confirm middleware headers still appear
You should still see:
- `X-Request-Id`
- access logs with request_id
- timing headers (if enabled)

---

## 27.16 Common Async Mistakes (and How to Fix Them)

### Mistake 1: “Async made my view slower”
Often because:
- you added async overhead without actual I/O concurrency benefit
- you still do mostly DB work and your DB is sync-limited
Fix:
- keep view sync unless you have real concurrent waits
- optimize ORM and indexes first

### Mistake 2: `SynchronousOnlyOperation`
Cause:
- calling sync-only Django operations in async context
Fix:
- use async ORM variants or `async for`
- move transactional work into sync functions wrapped with `sync_to_async`

### Mistake 3: Async stack but sync middleware negates benefit
Cause:
- sync-only middleware forces thread-per-request
Fix:
- remove/replace with async-compatible middleware where possible
- or accept that ASGI is mainly for compatibility until you clean the stack

### Mistake 4: Long-lived view doesn’t handle disconnect
Cause:
- ignoring `CancelledError`
Fix:
- catch and cleanup if needed, then re-raise

---

## 27.17 Exercises (Do These Before Continuing)

1. Create an async CBV endpoint that does concurrent `asyncio.sleep` tasks and returns duration.
2. Create a view that intentionally triggers `SynchronousOnlyOperation`, observe it, then fix it using:
   - async ORM method or async iteration, or
   - a sync island wrapped with `sync_to_async`.
3. Run the project under Uvicorn and write down:
   - which endpoints are async
   - whether your middleware stack is being adapted (enable `django.request` debug logs)
4. Add an async test using `AsyncClient` for one endpoint.

---

## 27.18 Chapter Summary

- Async views are supported; real async scalability needs ASGI.
- Async helps with **concurrent I/O**, not CPU-heavy work.
- Middleware compatibility matters: sync middleware can remove async advantages.
- Django ORM has async query methods/iteration, but **transactions still don’t work
  in async mode**, so use sync islands for transactional workflows.
- Use `sync_to_async` (with `thread_sensitive=True` when needed) to call sync code safely.
- Run under Uvicorn to validate your ASGI setup and async behavior.

---

Next chapter: **28. Realtime with WebSockets (Django Channels)**  
We’ll add a realtime feature (notifications/chat), introduce consumers, routing,
channel layers (Redis), authorization for websockets, and scaling patterns.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../5. Apis_with_Django/26. api_security_and_production_concerns.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='28. realtime_with_websocets.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
